<a href="https://colab.research.google.com/github/Aesop-programmer/ML/blob/main/%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 pip install pt-datasets

In [2]:
from pt_datasets import load_dataset
train_data, test_data = load_dataset(name="malimg")


In [15]:
import random
import pandas as pd
import numpy as np
myseed = 123
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [3]:
family = list()
for i in range(len(train_data)):
  if train_data[i][1] not in family:
    family.append(train_data[i][1])
print(len(family))
#總共有25個class

25


In [100]:
#先分出18個，剩下的7個來測驗model是否能挑出來，並將其分成為training和validation
#其輸入的dim為1024，將其改為32*32
import torch
from torch.utils.data import Dataset, DataLoader

class train_data_18(Dataset):
  def __init__(self,train_data_25):
    self.data = list()
    for i in range(len(train_data)):
      if train_data[i][1] < 18:
        self.data.append(((np.array(train_data[i][0],dtype=np.float32)).reshape(1,32,32),np.array((train_data[i][1]),dtype=np.long)))
  def __getitem__(self,index):
    return(self.data[index])
  def __len__(self):
    return len(self.data)

train_data_set = train_data_18(train_data)

class train_val(Dataset):
  def __init__(self,dataset,mode,part):
    self.data=list()
    if mode== 'train':
      for i in range(len(dataset)):
        if i % part != 0:
          self.data.append(dataset[i])
    elif mode=='validation':
      for i in range(len(dataset)):
        if i % part == 0:
          self.data.append(dataset[i])
  def __getitem__(self,index):
    return(self.data[index])
  def __len__(self):
    return len(self.data)
train = DataLoader(train_val(train_data_set,'train',10),256,shuffle=True,drop_last=False,pin_memory=True)
val = DataLoader(train_val(train_data_set,'validation',10),256,shuffle=False,drop_last=False,pin_memory=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # This is added back by InteractiveShellApp.init_path()


In [104]:
#測試時 為了知道模型是否能指認出沒看過的data，因此將剩餘的資料分為18個(已看過)與7個(沒看過)
class test_18(Dataset):
  def __init__(self,test_data):
    self.data = list()
    for i in range(len(test_data)):
      if test_data[i][1] < 18:
        self.data.append(((np.array(test_data[i][0],dtype=np.float32)).reshape(1,32,32),np.array((test_data[i][1]),dtype=np.long)))
  def __getitem__(self,index):
    return(self.data[index])
  def __len__(self):
    return len(self.data)
class test_7(Dataset):
  def __init__(self,test_data,train_data):
    self.data = list()
    for i in range(len(test_data)):
      if test_data[i][1] >= 18:
        self.data.append(((np.array(test_data[i][0],dtype=np.float32)).reshape(1,32,32),np.array((test_data[i][1]),dtype=np.long)))
    for i in range(len(train_data)):
      if train_data[i][1] >= 18:
        self.data.append(((np.array(train_data[i][0],dtype=np.float32)).reshape(1,32,32),np.array((train_data[i][1]),dtype=np.long)))
  def __getitem__(self,index):
    return(self.data[index])
  def __len__(self):
    return len(self.data)
test_18 = DataLoader(test_18(test_data),256,shuffle=False,drop_last=False)
test_7 = DataLoader(test_7(test_data,train_data),256,shuffle=False,drop_last=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the b

In [121]:
import torch.nn as nn
import torchvision.transforms as transforms

class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()
    self.cnn_layers =nn.Sequential(
        nn.Conv2d(1,36,5,1),
        nn.MaxPool2d(2,1,0),
        nn.Conv2d(36,72,5,1),
        nn.LeakyReLU(),
        nn.MaxPool2d(2,1,0),
    )
    self.fc_layers = nn.Sequential(
        nn.Linear(22*22*72,512),
        nn.ReLU(),
        nn.Linear(512,256),
        nn.ReLU(),
        nn.Linear(256,18),
    )
  def forward(self,x):
    #因為input為(batchsize,1024)，我們須將其轉為32x32
    x = self.cnn_layers(x)
    #將其拉為一條，在放入fully_connected layer
    x = x.flatten(1)
    x = self.fc_layers(x)
    return x

In [122]:
#開始訓練前的設定
device = "cuda" if torch.cuda.is_available() else "cpu"

n_epochs = 10
patience = 300 #for early stop
model = Classifier().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(),lr=1e-3,weight_decay=1e-5)


In [123]:
from tqdm.auto import tqdm
stale = 0
best_acc = 0

for epoch in range(n_epochs):
  #training
  model.train()
  train_loss =[]
  train_accs = []
  for batch in tqdm(train):

    imgs,labels = batch
    
    logits = model(imgs.to(device))

    loss =criterion(logits,labels.to(device))

    optimizer.zero_grad()

    loss.backward()

    #確保gradient descent 穩定
    grad_norm = nn.utils.clip_grad_norm_(model.parameters(),max_norm=10)

    optimizer.step()
    acc = (logits.argmax(dim=-1)==labels.to(device)).float().mean()

    train_loss.append(loss.item())
    train_accs.append(acc)
  train_loss = sum(train_loss) / len(train_loss)
  train_acc = sum(train_accs) / len(train_accs)
  print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

  #validation
  model.eval()

  valid_loss = []
  valid_accs = []

  for batch in tqdm(val):
    imgs, labels = batch

    with torch.no_grad():
      logits = model(imgs.to(device))

    loss = criterion(logits,labels.to(device))
    
    acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

    valid_loss.append(loss.item())
    valid_accs.append(acc)
  valid_loss = sum(valid_loss) / len(valid_loss)
  valid_acc = sum(valid_accs) / len(valid_accs)
  print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

  #存model
  if valid_acc > best_acc:
    print(f"best model found as epoch {epoch},acc is {valid_acc}")
    torch.save(model.state_dict(),f"cnn_best.ckpt")
    best_acc = valid_acc
    stale = 0
  else:
    stale +=1
    if stale > patience:
      print(f"no imporvement {patience} consecutive epochs, early stopping")
      break

  0%|          | 0/19 [00:00<?, ?it/s]

[ Train | 001/010 ] loss = 9.36209, acc = 0.36141


  0%|          | 0/3 [00:00<?, ?it/s]

[ Valid | 001/010 ] loss = 0.90928, acc = 0.55237
best model found as epoch 0,acc is 0.5523726940155029


  0%|          | 0/19 [00:00<?, ?it/s]

...

[ Valid | 009/010 ] loss = 0.07723, acc = 0.98438
best model found as epoch 8,acc is 0.984375


  0%|          | 0/19 [00:00<?, ?it/s]

[ Train | 010/010 ] loss = 0.06855, acc = 0.97661


  0%|          | 0/3 [00:00<?, ?it/s]

[ Valid | 010/010 ] loss = 0.06727, acc = 0.98698
best model found as epoch 9,acc is 0.9869791865348816


In [124]:
import math
model.eval()
prediction =[] #用來確認模型在已知class的精準度

prediction_18 =[] #用來算已知class的entropy
prediction_7=[] #用來算未知class的entropy

for batch in tqdm(test_18):
  imgs, labels = batch
  with torch.no_grad():
    logits = model(imgs.to(device))
  acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
  for i in range((len(logits))):
    entropy = 0
    #換成sofmax 再來計算entropy
    logits[i] = np.exp(logits[i])/sum(np.exp(logits[i]))
    for j in range(len(logits[0])):
      entropy = entropy + (-logits[0][j]*(math.log(logits[0][j])))
      prediction_18.append(entropy)
  prediction.append(acc)
print(f"已知class的精準度{sum(prediction)/len(prediction)}")

for batch in tqdm(test_7):
  imgs, labels = batch
  with torch.no_grad():
    logits = model(imgs.to(device))
  for i in range((len(logits))):
    entropy = 0
    logits[i] = np.exp(logits[i])/sum(np.exp(logits[i]))
    for j in range(len(logits[0])):
      entropy = entropy + (-logits[0][j]*(math.log(logits[0][j])))
      prediction_7.append(entropy)


  0%|          | 0/9 [00:00<?, ?it/s]

已知class的精準度0.9652226567268372


  0%|          | 0/8 [00:00<?, ?it/s]

In [127]:
print(sum(prediction_7)/len(prediction_7))
print(sum(prediction_18)/len(prediction_18))

tensor(0.1346)
tensor(0.0636)
